In [22]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler



In [23]:
dataset_orig_panel19_train = MEPSDataset19()

In [3]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [4]:
dataset_orig_panel19_train.feature_names[1]

'RACE'

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare data
X = dataset_orig_panel19_train.features
y = dataset_orig_panel19_train.labels.ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Predict probabilities on the same scaled training data
train_probabilities = model.predict_proba(X_train_scaled)[:, 1]

# Calculation of discrimination index without modifying dataset structure
sens_attr_index = dataset_orig_panel19_train.feature_names.index('RACE')



# Define unprivileged and privileged values
unprivileged_val = 0.0
privileged_val = 1.0



c:\Users\srinivas\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Discrimination Loss Fn

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function
def discrimination_loss(output, target, sensitive_features, lambda_val=10, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)
    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])

    discrimination = lambda_val * (prob_priv - prob_unpriv) ** k

    loss_val=(1 + lambda_val * discrimination) * standard_loss
    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model2 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model2.parameters(), lr=0.01)
train_losses, train_accuracies, train_discriminations,train_fairness = [], [], [],[]
test_losses, test_accuracies, test_discriminations,test_fairness = [], [], [],[]

# Training loop
model2.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model2(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model2.eval()
    with torch.no_grad():
        test_outputs = model2(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses.append(loss.item())
    train_accuracies.append(train_accuracy.item() * 100)
    train_discriminations.append(discrimination)
    fairness=1-discrimination
    train_fairness.append(fairness)
    test_fairness.append(fairness)
    test_losses.append(test_loss.item())
    test_accuracies.append(test_accuracy.item() * 100)
    test_discriminations.append(test_discrimination)
    model2.train()


Epoch 1, Train Loss: 1.9220411777496338, Train Acc: 18.66%, Train Discrimination: 0.02425457537174225 Test Loss: 1.1295995712280273, Test Acc: 60.01%,Test Discrimination: 0.06222127005457878
Epoch 2, Train Loss: 0.9532933831214905, Train Acc: 60.45%, Train Discrimination: 0.03780997917056084 Test Loss: 1.1869919300079346, Test Acc: 81.08%,Test Discrimination: 0.04194371774792671
Epoch 3, Train Loss: 1.027151346206665, Train Acc: 81.63%, Train Discrimination: 0.02452375739812851 Test Loss: 1.0255143642425537, Test Acc: 82.66%,Test Discrimination: 0.022671611979603767
Epoch 4, Train Loss: 0.9296919703483582, Train Acc: 82.87%, Train Discrimination: 0.013031406328082085 Test Loss: 0.8804137110710144, Test Acc: 82.66%,Test Discrimination: 0.012577864341437817
Epoch 5, Train Loss: 0.8239301443099976, Train Acc: 82.87%, Train Discrimination: 0.007192534394562244 Test Loss: 0.7522690296173096, Test Acc: 82.66%,Test Discrimination: 0.00728579331189394
Epoch 6, Train Loss: 0.7177369594573975, T

#### MODEL ACCROSS DIFFERENT VALUES OF LAMBDA

#### LAMBDA < 1

In [7]:
model3 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model3.parameters(), lr=0.01)
train_losses3, train_accuracies3, train_discriminations3,train_fairness3 = [], [], [],[]
test_losses3, test_accuracies3, test_discriminations3,test_fairness3 = [], [], [],[]

# Training loop
model3.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model3(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=0.01, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model3.eval()
    with torch.no_grad():
        test_outputs = model3(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=0.01, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses3.append(loss.item())
    train_accuracies3.append(train_accuracy.item() * 100)
    train_discriminations3.append(discrimination)
    fairness=1-discrimination
    train_fairness3.append(fairness)
    test_fairness3.append(fairness)
    test_losses3.append(test_loss.item())
    test_accuracies3.append(test_accuracy.item() * 100)
    test_discriminations3.append(test_discrimination)
    model3.train()

Epoch 1, Train Loss: 1.0041264295578003, Train Acc: 17.56%, Train Discrimination: 1.978987165784929e-05 Test Loss: 0.6168790459632874, Test Acc: 82.69%,Test Discrimination: 1.787938890629448e-05
Epoch 2, Train Loss: 0.6103816032409668, Train Acc: 82.77%, Train Discrimination: 1.0953659511869773e-05 Test Loss: 0.6829957365989685, Test Acc: 82.66%,Test Discrimination: 6.36765935269068e-06
Epoch 3, Train Loss: 0.6712662577629089, Train Acc: 82.87%, Train Discrimination: 3.88531543649151e-06 Test Loss: 0.6398752927780151, Test Acc: 82.66%,Test Discrimination: 9.294354867961374e-07
Epoch 4, Train Loss: 0.6270580291748047, Train Acc: 82.87%, Train Discrimination: 4.72430514264488e-07 Test Loss: 0.5545927882194519, Test Acc: 82.66%,Test Discrimination: 5.3819807277477594e-08
Epoch 5, Train Loss: 0.5427716970443726, Train Acc: 82.87%, Train Discrimination: 1.1849394354612741e-07 Test Loss: 0.4637513756752014, Test Acc: 82.66%,Test Discrimination: 2.6729439923656173e-06
Epoch 6, Train Loss: 0.4

#### LAMBDA = 1

In [8]:
model4 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model4.parameters(), lr=0.01)
train_losses4, train_accuracies4, train_discriminations4,train_fairness4 = [], [], [],[]
test_losses4, test_accuracies4, test_discriminations4,test_fairness4 = [], [], [],[]

# Training loop
model4.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model4(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=1, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model4.eval()
    with torch.no_grad():
        test_outputs = model4(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=1, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses4.append(loss.item())
    train_accuracies4.append(train_accuracy.item() * 100)
    train_discriminations4.append(discrimination)
    fairness=1-discrimination
    train_fairness4.append(fairness)
    test_fairness4.append(fairness)
    test_losses4.append(test_loss.item())
    test_accuracies4.append(test_accuracy.item() * 100)
    test_discriminations4.append(test_discrimination)
    model4.train()


Epoch 1, Train Loss: 1.8446720838546753, Train Acc: 17.13%, Train Discrimination: 0.001698514330200851 Test Loss: 0.524501621723175, Test Acc: 82.19%,Test Discrimination: 0.0006768041639588773
Epoch 2, Train Loss: 0.5233855843544006, Train Acc: 82.25%, Train Discrimination: 0.00046851675142534077 Test Loss: 0.7035712599754333, Test Acc: 82.66%,Test Discrimination: 0.00064799067331478
Epoch 3, Train Loss: 0.6930978298187256, Train Acc: 82.87%, Train Discrimination: 0.0004053693264722824 Test Loss: 0.7174410223960876, Test Acc: 82.66%,Test Discrimination: 0.00016288572805933654
Epoch 4, Train Loss: 0.7047736048698425, Train Acc: 82.87%, Train Discrimination: 9.914215479511768e-05 Test Loss: 0.6332312822341919, Test Acc: 82.66%,Test Discrimination: 1.980807792278938e-05
Epoch 5, Train Loss: 0.6210913062095642, Train Acc: 82.87%, Train Discrimination: 8.91736908670282e-06 Test Loss: 0.5379812717437744, Test Acc: 82.66%,Test Discrimination: 1.009002880891785e-05
Epoch 6, Train Loss: 0.52786

#### LAMBDA > 1

In [9]:
model5 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model5.parameters(), lr=0.01)
train_losses5, train_accuracies5, train_discriminations5,train_fairness5 = [], [], [],[]
test_losses5, test_accuracies5, test_discriminations5,test_fairness5 = [], [], [],[]

# Training loop
model5.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model5(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=10, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model5.eval()
    with torch.no_grad():
        test_outputs = model5(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=10, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses5.append(loss.item())
    train_accuracies5.append(train_accuracy.item() * 100)
    train_discriminations5.append(discrimination)
    fairness=1-discrimination
    train_fairness5.append(fairness)
    test_fairness5.append(fairness)
    test_losses5.append(test_loss.item())
    test_accuracies5.append(test_accuracy.item() * 100)
    test_discriminations5.append(test_discrimination)
    model5.train()


Epoch 1, Train Loss: 0.6827158331871033, Train Acc: 68.75%, Train Discrimination: 0.01587730087339878 Test Loss: 2.0852560997009277, Test Acc: 43.84%,Test Discrimination: 0.08393824845552444
Epoch 2, Train Loss: 1.8346697092056274, Train Acc: 44.15%, Train Discrimination: 0.060856565833091736 Test Loss: 0.5064712166786194, Test Acc: 84.11%,Test Discrimination: 0.003421199042350054
Epoch 3, Train Loss: 0.515932023525238, Train Acc: 84.86%, Train Discrimination: 0.0057143233716487885 Test Loss: 0.7100622653961182, Test Acc: 82.66%,Test Discrimination: 0.017768755555152893
Epoch 4, Train Loss: 0.6470770239830017, Train Acc: 82.87%, Train Discrimination: 0.009518122300505638 Test Loss: 0.881696879863739, Test Acc: 82.66%,Test Discrimination: 0.017642173916101456
Epoch 5, Train Loss: 0.8068544268608093, Train Acc: 82.87%, Train Discrimination: 0.010149571113288403 Test Loss: 0.8861703872680664, Test Acc: 82.66%,Test Discrimination: 0.01256042905151844
Epoch 6, Train Loss: 0.8244614601135254

#### LAMBDA >> 1

In [10]:
model6 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model6.parameters(), lr=0.01)
train_losses6, train_accuracies6, train_discriminations6,train_fairness6 = [], [], [],[]
test_losses6, test_accuracies6, test_discriminations6,test_fairness6 = [], [], [],[]

# Training loop
model6.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model6(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model6.eval()
    with torch.no_grad():
        test_outputs = model6(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses6.append(loss.item())
    train_accuracies6.append(train_accuracy.item() * 100)
    train_discriminations6.append(discrimination)
    fairness=1-discrimination
    train_fairness6.append(fairness)
    test_fairness6.append(fairness)
    test_losses6.append(test_loss.item())
    test_accuracies6.append(test_accuracy.item() * 100)
    test_discriminations6.append(test_discrimination)
    model6.train()



Epoch 1, Train Loss: 0.7337751388549805, Train Acc: 62.83%, Train Discrimination: 0.0008777288603596389 Test Loss: 147.70858764648438, Test Acc: 50.60%,Test Discrimination: 0.9427163600921631
Epoch 2, Train Loss: 97.17446899414062, Train Acc: 50.56%, Train Discrimination: 0.6176936626434326 Test Loss: 13.705608367919922, Test Acc: 51.23%,Test Discrimination: 0.17300979793071747
Epoch 3, Train Loss: 11.584554672241211, Train Acc: 51.69%, Train Discrimination: 0.1452496200799942 Test Loss: 24.729219436645508, Test Acc: 64.18%,Test Discrimination: 0.38477006554603577
Epoch 4, Train Loss: 14.460345268249512, Train Acc: 64.88%, Train Discrimination: 0.2236996293067932 Test Loss: 42.92346954345703, Test Acc: 62.95%,Test Discrimination: 0.562828004360199
Epoch 5, Train Loss: 25.520246505737305, Train Acc: 63.80%, Train Discrimination: 0.33555302023887634 Test Loss: 41.004459381103516, Test Acc: 69.87%,Test Discrimination: 0.5177189111709595
Epoch 6, Train Loss: 24.27086067199707, Train Acc: 7

#### MODEL ACCROSS DIFFERENT VALUES OF K

### K=2

In [11]:
model7 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model7.parameters(), lr=0.01)
train_losses7, train_accuracies7, train_discriminations7,train_fairness7 = [], [], [],[]
test_losses7, test_accuracies7, test_discriminations7,test_fairness7 = [], [], [],[]

# Training loop
model7.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model7(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=2)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model7.eval()
    with torch.no_grad():
        test_outputs = model7(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=2)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses7.append(loss.item())
    train_accuracies7.append(train_accuracy.item() * 100)
    train_discriminations7.append(discrimination)
    fairness=1-discrimination
    train_fairness7.append(fairness)
    test_fairness7.append(fairness)
    test_losses7.append(test_loss.item())
    test_accuracies7.append(test_accuracy.item() * 100)
    test_discriminations7.append(test_discrimination)
    model7.train()




Epoch 1, Train Loss: 11.502778053283691, Train Acc: 82.87%, Train Discrimination: 0.17301660776138306 Test Loss: 97.62801361083984, Test Acc: 50.85%,Test Discrimination: 0.9923915266990662
Epoch 2, Train Loss: 74.85464477539062, Train Acc: 50.84%, Train Discrimination: 0.7589302062988281 Test Loss: 1.8448865413665771, Test Acc: 82.66%,Test Discrimination: 0.02714366465806961
Epoch 3, Train Loss: 0.9399725794792175, Train Acc: 82.87%, Train Discrimination: 0.009198581799864769 Test Loss: 12.709319114685059, Test Acc: 82.66%,Test Discrimination: 0.16618643701076508
Epoch 4, Train Loss: 7.349886417388916, Train Acc: 82.87%, Train Discrimination: 0.09380106627941132 Test Loss: 15.005339622497559, Test Acc: 82.66%,Test Discrimination: 0.15238988399505615
Epoch 5, Train Loss: 8.770332336425781, Train Acc: 82.87%, Train Discrimination: 0.08679691702127457 Test Loss: 12.97114086151123, Test Acc: 82.66%,Test Discrimination: 0.11009559780359268
Epoch 6, Train Loss: 7.61027717590332, Train Acc: 8

### K=3

In [12]:
model8 = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model8.parameters(), lr=0.01)
train_losses8, train_accuracies8, train_discriminations8,train_fairness8 = [], [], [],[]
test_losses8, test_accuracies8, test_discriminations8,test_fairness8 = [], [], [],[]

# Training loop
model8.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model8(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features, lambda_val=100, k=3)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model8.eval()
    with torch.no_grad():
        test_outputs = model8(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features, lambda_val=100, k=3)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses8.append(loss.item())
    train_accuracies8.append(train_accuracy.item() * 100)
    train_discriminations8.append(discrimination)
    fairness=1-discrimination
    train_fairness8.append(fairness)
    test_fairness8.append(fairness)
    test_losses8.append(test_loss.item())
    test_accuracies8.append(test_accuracy.item() * 100)
    test_discriminations8.append(test_discrimination)
    model8.train()





Epoch 1, Train Loss: 1.7789134979248047, Train Acc: 17.13%, Train Discrimination: 0.005894949659705162 Test Loss: -11.303778648376465, Test Acc: 57.71%,Test Discrimination: -0.13165916502475739
Epoch 2, Train Loss: -5.359508037567139, Train Acc: 57.98%, Train Discrimination: -0.06815020740032196 Test Loss: -37.331298828125, Test Acc: 56.79%,Test Discrimination: -0.2463148832321167
Epoch 3, Train Loss: -17.097667694091797, Train Acc: 56.61%, Train Discrimination: -0.1192392110824585 Test Loss: -82.96578216552734, Test Acc: 52.65%,Test Discrimination: -0.364091157913208
Epoch 4, Train Loss: -38.91627883911133, Train Acc: 52.60%, Train Discrimination: -0.1773332953453064 Test Loss: -136.23008728027344, Test Acc: 49.43%,Test Discrimination: -0.4250484108924866
Epoch 5, Train Loss: -64.56844329833984, Train Acc: 49.67%, Train Discrimination: -0.20796793699264526 Test Loss: -193.1113739013672, Test Acc: 49.37%,Test Discrimination: -0.4506971836090088
Epoch 6, Train Loss: -91.29541778564453, 

#### MODEL ACCROSS DIFFERENT VALUES OF LAMBDA

In [13]:
import plotly.graph_objects as go

def plot_comp_metric(title, y_label, val1, val2, val3, val4):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    if val1:
        fig.add_trace(go.Scatter(
            x=epochs, y=val1, mode='lines+markers',
            name='Lambda = 0.1',
            line=dict(color='RoyalBlue', width=2),
            marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    if val2:

        # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val2, mode='lines+markers',
            name='lambda = 1',
            line=dict(color='Crimson', width=2, dash='dot'),
            marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    # Adding Test Line with Markers
    if val3:
        fig.add_trace(go.Scatter(
            x=epochs, y=val3, mode='lines+markers',
            name='lambda = 10',
            line=dict(color='Pink', width=2, dash='dot'),
            marker=dict(color='Pink', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))

    if val4:
     # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val4, mode='lines+markers',
            name='lambda = 100',
            line=dict(color='Purple', width=2, dash='dot'),
            marker=dict(color='Purple', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))


    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [14]:
plot_comp_metric("TEST_Fairness", "Fairness", test_fairness3, test_fairness4, test_fairness5, test_fairness6)
plot_comp_metric("TEST_Discriminations", "Discrimination", test_discriminations3, test_discriminations4, test_discriminations5, test_discriminations6)
plot_comp_metric("TEST_Losses", "Losses", test_losses3,test_losses4,test_losses5,test_losses6)


#### MODEL ACCROSS DIFFERENT VALUES OF K

In [15]:
import plotly.graph_objects as go

def plot_comp_metric_k(title, y_label, val1, val2):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    if val1:
        fig.add_trace(go.Scatter(
            x=epochs, y=val1, mode='lines+markers',
            name='K = 2',
            line=dict(color='RoyalBlue', width=2),
            marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))
    if val2:

        # Adding Test Line with Markers
        fig.add_trace(go.Scatter(
            x=epochs, y=val2, mode='lines+markers',
            name='K = 3',
            line=dict(color='Crimson', width=2, dash='dot'),
            marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
        ))



    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [16]:
plot_comp_metric_k("TEST_Fairness", "Fairness", test_fairness7, test_fairness8)
plot_comp_metric_k("TEST_Discriminations", "Discrimination", test_discriminations7, test_discriminations8)
plot_comp_metric_k("TEST_Losses", "Losses", test_losses7,test_losses8)

#### LOSS ACCURACY AND DISCRIMINATION FOR MODEL WITH DISCRIMINATION FUNCTION

In [17]:
import plotly.graph_objects as go

def plot_metric(title, y_label, train_data, test_data):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name='Train',
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name='Test',
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title='Epoch',
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

In [18]:
# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses, test_losses)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies, test_accuracies)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations, test_discriminations)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness, test_fairness)

## BCE Loss Fn

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Custom loss function|
def discrimination_loss(output, target, sensitive_features, lambda_val=100, k=2):
    criterion = nn.BCELoss()
    standard_loss = criterion(output, target)

    mask_unpriv = (sensitive_features == 0)
    mask_priv = (sensitive_features == 1)
    #discrimination=torch.abs(prob_priv)
    prob_unpriv = torch.mean(output[mask_unpriv])
    prob_priv = torch.mean(output[mask_priv])
    discrimination = lambda_val*(prob_priv - prob_unpriv) ** k

# Handle cases where one group might be missing
    #discrimination=torch.abs(prob_priv)
 
    loss_val=standard_loss

    return loss_val,discrimination.item() 

def calculate_accuracy(predictions, targets):
    predicted_classes = (predictions >= 0.5).float()
    return (predicted_classes == targets).float().mean()


data = torch.tensor(X_train_scaled).float()
targets = torch.tensor(y_train).float().unsqueeze(1)

# Correctly preparing the sensitive features
threshold = 0.5  # Adjust the threshold according to your specific case
sensitive_features = torch.tensor((data[:, 1].numpy() > threshold).astype(float)).float()
features = torch.cat((data[:, :1], data[:, 2:]), dim=1)

# Assuming similar preparation for test data
test_data = torch.tensor(X_test_scaled).float()
test_targets = torch.tensor(y_test).float().unsqueeze(1)
test_sensitive_features = torch.tensor((test_data[:, 1].numpy() > threshold).astype(float)).float()
test_features = torch.cat((test_data[:, :1], test_data[:, 2:]), dim=1)

model = BinaryClassifier(features.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)
train_losses1, train_accuracies1, train_discriminations1,train_fairness1 = [], [], [],[]
test_losses1, test_accuracies1, test_discriminations1,test_fairness1 = [], [], [],[]
# Training loop
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(features)
    loss, discrimination = discrimination_loss(outputs, targets.squeeze(), sensitive_features)
    train_accuracy = calculate_accuracy(outputs, targets.squeeze())
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data
    model.eval()
    with torch.no_grad():
        test_outputs = model(test_features)
        test_loss,test_discrimination = discrimination_loss(test_outputs, test_targets.squeeze(), test_sensitive_features)
        test_accuracy = calculate_accuracy(test_outputs, test_targets.squeeze())
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item()}, Train Acc: {train_accuracy.item()*100:.2f}%, Train Discrimination: {discrimination} '
          f'Test Loss: {test_loss.item()}, Test Acc: {test_accuracy.item()*100:.2f}%,Test Discrimination: {test_discrimination}')
    train_losses1.append(loss.item())
    train_accuracies1.append(train_accuracy.item() * 100)
    train_discriminations1.append(discrimination)
    fairness=1-discrimination
    train_fairness1.append(fairness)
    test_fairness1.append(fairness)
    test_losses1.append(test_loss.item())
    test_accuracies1.append(test_accuracy.item() * 100)
    test_discriminations1.append(test_discrimination)
    model.train()


Epoch 1, Train Loss: 1.1222906112670898, Train Acc: 17.27%, Train Discrimination: 0.10794885456562042 Test Loss: 0.6396477222442627, Test Acc: 82.66%,Test Discrimination: 0.2171773761510849
Epoch 2, Train Loss: 0.6310044527053833, Train Acc: 82.87%, Train Discrimination: 0.13623325526714325 Test Loss: 0.7794495820999146, Test Acc: 82.66%,Test Discrimination: 0.08968199789524078
Epoch 3, Train Loss: 0.7643031477928162, Train Acc: 82.87%, Train Discrimination: 0.05449695885181427 Test Loss: 0.767382025718689, Test Acc: 82.66%,Test Discrimination: 0.03229247406125069
Epoch 4, Train Loss: 0.7517351508140564, Train Acc: 82.87%, Train Discrimination: 0.019562693312764168 Test Loss: 0.6684368848800659, Test Acc: 82.66%,Test Discrimination: 0.009146862663328648
Epoch 5, Train Loss: 0.6537835597991943, Train Acc: 82.87%, Train Discrimination: 0.005255941767245531 Test Loss: 0.5339722633361816, Test Acc: 82.66%,Test Discrimination: 6.4282298808393534e-06
Epoch 6, Train Loss: 0.5220515727996826, 

In [20]:
import plotly.graph_objects as go


def plot_metric(title, y_label, train_data, test_data, x_title = "Epoch", epoch_blue = "train", epoch_red = "Test"):
    epochs = list(range(1, 101))
    fig = go.Figure()

    # Adding Train Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=train_data, mode='lines+markers',
        name=epoch_blue,
        line=dict(color='RoyalBlue', width=2),
        marker=dict(color='RoyalBlue', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Adding Test Line with Markers
    fig.add_trace(go.Scatter(
        x=epochs, y=test_data, mode='lines+markers',
        name=epoch_red,
        line=dict(color='Crimson', width=2, dash='dot'),
        marker=dict(color='Crimson', size=6, line=dict(width=1, color='DarkSlateGrey'))
    ))

    # Update Layout
    fig.update_layout(
        title={'text': title, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        xaxis_title=x_title,
        yaxis_title=y_label,
        legend=dict(x=0.1, y=1.1, orientation='h'),
        font=dict(family="Helvetica, Arial, sans-serif", size=12, color="black"),
        plot_bgcolor='white',
        margin=dict(l=40, r=40, t=40, b=30)
    )

    # Gridlines and Axes styles
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='LightGrey')

    fig.show()

# Example usage
plot_metric("Training and Testing Loss", "Loss", train_losses1, test_losses1)
plot_metric("Training and Testing Accuracy", "Accuracy (%)", train_accuracies1, test_accuracies1)
plot_metric("Training and Testing Discrimination", "Discrimination", train_discriminations1, test_discriminations1)
plot_metric("Training and Testing Fairness", "Fairness", train_fairness1, test_fairness1)



In [21]:
plot_metric("DISPARITY IN FAIRNESS", "FAIRNESS", test_fairness, test_fairness1, "EPOCH", "WITH DISCRIMINATION FUNCTION", "BCE")